## Dashboard Climate-Trace 

Showing plots about climate-trace data.

In [1]:
import pandas as pd
import os; os.chdir('../') # to run from repo root
import sys; sys.path.insert(0, '/Users/aprevot/Synced/github/code/code_perso/yaetos/')  # to run from git repo, not lib. need to reload jupyter kernel for change to take effect.
from yaetos.etl_utils import InputLoader

args = {
    'job_param_file': 'conf/jobs_metadata.yml',
    'job_name': 'dashboards/climate_trace.ipynb',
    'py_job': 'n/a',
    'dependencies': False,
    }

inputs = InputLoader(**args).run()
assets = inputs['assets']
assets.tail()

|13:43:14:INFO:Job|Path to library file (to know if running from yaetos lib or git repo): /Users/aprevot/Synced/github/code/code_perso/yaetos/yaetos/etl_utils.py
|13:43:14:INFO:Job|Job args: 
{'add_created_at': 'true',
 'aws_config_file': 'none',
 'aws_setup': 'dev',
 'base_path': '{root_path}/data',
 'chain_dependencies': False,
 'code_source': 'lib',
 'connection_file': 'conf/connections.cfg',
 'dependencies': False,
 'deploy': 'none',
 'description': 'Dashboard to visualize climate_trace data.',
 'email_cred_section': 'some_email_cred_section',
 'enable_redshift_push': False,
 'inputs': {'assets': {'df_type': 'pandas',
                       'path': '{base_path}/climate_trace/assets_master_california/{latest}/dataset.csv',
                       'type': 'csv'}},
 'is_incremental': False,
 'job_name': 'dashboards/climate_trace.ipynb',
 'job_param_file': 'conf/jobs_metadata.yml',
 'jobs_folder': 'jobs/',
 'leave_on': False,
 'load_connectors': 'none',
 'manage_git_info': False,
 'mode

,Unnamed: 0.1,Unnamed: 0,Id,Name,NativeId,Country,Sector,AssetType,ReportingEntity,Centroid,offset_batch,latitude,longitude,Thumbnail_post,co2
5305,5305,46567,3265253,RICHVALE WWTF,USA01144,USA,wastewater-treatment-and-discharge,domestic centralized aerobic secondary,climate-trace,"{'Geometry': array([-122.081, 39.386]), 'SRI...",46500,39.386000,-122.081000,https://api.mapbox.com/v4/mapbox.satellite/16/...,0.0
5306,5306,21012,25745436,USA_CA_dairy_1248,USA_CA_dairy_1248,USA,manure-management-cattle-feedlot,manure_management_dairy,climate-trace,"{'Geometry': array([-119.71114531, 36.222698...",21000,36.222699,-119.711145,https://api.mapbox.com/v4/mapbox.satellite/16/...,0.0
5307,5307,24557,3160415,USA_CA_dairy_354,USA_CA_dairy_354,USA,enteric-fermentation-cattle-feedlot,dairy,climate-trace,"{'Geometry': array([-121.018 , 37.80671]), ...",24500,37.806710,-121.018000,https://api.mapbox.com/v4/mapbox.satellite/16/...,0.0
5308,5308,24670,3161537,USA_CA_dairy_1476,USA_CA_dairy_1476,USA,enteric-fermentation-cattle-feedlot,dairy,climate-trace,"{'Geometry': array([-121.00264633, 37.158295...",24600,37.158296,-121.002646,https://api.mapbox.com/v4/mapbox.satellite/16/...,0.0
5309,5309,17536,3161534,USA_CA_dairy_1473,USA_CA_dairy_1473,USA,enteric-fermentation-cattle-feedlot,dairy,climate-trace,"{'Geometry': array([-119.31248711, 36.150775...",17500,36.150776,-119.312487,https://api.mapbox.com/v4/mapbox.satellite/16/...,0.0


In [2]:
import panel as pn
import pydeck as pdk
pn.extension('deckgl')
pn.extension('tabulator')

In [3]:
import pydeck as pdk

CLIMATE_TRACE_DATA = assets[['Name', 'longitude', 'latitude', "co2"]]

# Define a layer to display on a map
layer = pdk.Layer(
    'HexagonLayer',
    CLIMATE_TRACE_DATA,
    get_position=['longitude', 'latitude'],
    # get_elevation="co2",
    # get_tooltip="Name: {Name}",
    auto_highlight=True,
    elevation_scale=50,
    pickable=True,
    elevation_range=[0, 3000],
    extruded=True,                 
    coverage=1)

# Set the viewport location
view_state = pdk.ViewState(
    longitude=-120.415,
    latitude=37.2323,
    zoom=5,  # lower number is seeing from higher
    min_zoom=3,
    max_zoom=15,
    pitch=40.5,
    # bearing=-27.36)
    bearing=-0.36)

# Render
deck = pdk.Deck(layers=[layer], initial_view_state=view_state)
# deck.to_html('./dashboards/demo_standalone.html')

# NOTE: doesn't require the mapbox API key. 

In [4]:
# Build box_assets
box_assets = pn.indicators.Number(name="Emitting assets", value=len(assets), font_size='15pt', default_color='#3498db')
box_assets

BokehModel(combine_events=True, render_bundle={'docs_json': {'aeccd0c6-5807-405a-a638-a8752050371d': {'version…

In [5]:
# Build box_emissions
import humanize
# def format_nb(value):
#     return f"{value:.2%}"
#emissions = humanize.intword(inputs['assets']['co2'].sum())
emissions = assets['co2'].sum()
#box_emissions = pn.indicators.Number(name="Carbon Emissions Total", value=emissions, format="{,.2f}") # doesn't work
box_emissions = pn.indicators.Number(name="Carbon Emissions", value=emissions, format="{value:,.0f}", font_size='15pt', default_color='#3498db') # works
# box_emissions = pn.indicators.Number(name="Carbon Emissions Total", value=emissions, format=format_nb) # doesn't work
box_emissions

BokehModel(combine_events=True, render_bundle={'docs_json': {'2f33b72f-2cca-4209-b9c6-d181dc0ab37b': {'version…

In [6]:
# Build table
tableUI = pn.widgets.Tabulator(assets[['Name', 'co2', 'Sector', 'AssetType', 'latitude', 'longitude', 'Thumbnail_post']], height=200, sizing_mode="stretch_width")
tableUI

BokehModel(combine_events=True, render_bundle={'docs_json': {'0640bbba-7f5a-4d67-94f1-2a6d318294d7': {'version…

In [7]:
# Content

content_row0 = pn.panel("Climate Trace Dashboard", align="center", styles={"background-color": "#3498db", "color": "white", "width": "100%"})
content_row1 = pn.pane.Markdown("This shows carbon emissions associated to specific locations (assets) in California. \nThis is Work In Progress ! Demo only...")
content_row2_column1_row1 = pn.pane.DeckGL(deck.to_json(), sizing_mode='stretch_width', height=500)
content_row2_column1_row2 = pn.pane.Markdown("Pan and zoom in map above")
content_row2_column2_row1 = pn.pane.Markdown("General Metrics")
content_row2_column2_row2 = box_assets
content_row2_column2_row3 = box_emissions
content_row3 = pn.pane.Markdown("#### List of emitting assets")
content_row4 = tableUI
content_row5 = pn.panel("Footer", align="center", styles={"background-color": "#3498db", "color": "white", "width": "100%"})


In [8]:
# Layout
# Read from the bottom of code to the top to understand the dashboard from top to bottom 
dash_row2_column1 = pn.Column(content_row2_column1_row1, content_row2_column1_row2)
dash_row2_column2 = pn.Column(content_row2_column2_row1, content_row2_column2_row2, content_row2_column2_row3)
dash_row2 = pn.Row(dash_row2_column1, dash_row2_column2)
dash = pn.Column(content_row0, content_row1, dash_row2, content_row3, content_row4, content_row5, sizing_mode="stretch_both")
dash.servable()
dash  # to display it
# dash.show() # to open in browser

BokehModel(combine_events=True, render_bundle={'docs_json': {'53553fec-4b55-417c-bebc-ea6cc9835e7c': {'version…

In [ ]:
dash.show() # to open in browser

|13:44:33:INFO:bokeh.server.server|Starting Bokeh server version 3.3.2 (running on Tornado 6.2)
|13:44:33:INFO:bokeh.server.tornado|User authentication hooks NOT provided (default user enabled)


Launching server at http://localhost:52753


|13:44:33:INFO:tornado.access|200 GET / (::1) 121.95ms
INFO:tornado.access:200 GET / (::1) 121.95ms
|13:44:33:INFO:tornado.access|200 GET /static/extensions/panel/bundled/deckglplot/mapbox-gl-js/v3.0.1/mapbox-gl.css (::1) 2.30ms
INFO:tornado.access:200 GET /static/extensions/panel/bundled/deckglplot/mapbox-gl-js/v3.0.1/mapbox-gl.css (::1) 2.30ms
|13:44:33:INFO:tornado.access|200 GET /static/extensions/panel/bundled/deckglplot/@loaders.gl/json@3.1.7/dist/dist.min.js (::1) 2.56ms
INFO:tornado.access:200 GET /static/extensions/panel/bundled/deckglplot/@loaders.gl/json@3.1.7/dist/dist.min.js (::1) 2.56ms
|13:44:33:INFO:tornado.access|200 GET /static/extensions/panel/bundled/deckglplot/@loaders.gl/csv@3.1.7/dist/dist.min.js (::1) 5.07ms
INFO:tornado.access:200 GET /static/extensions/panel/bundled/deckglplot/@loaders.gl/csv@3.1.7/dist/dist.min.js (::1) 5.07ms
|13:44:33:INFO:tornado.access|200 GET /static/extensions/panel/bundled/deckglplot/@loaders.gl/3d-tiles@3.1.7/dist/dist.min.js (::1) 3.